In [127]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from sklearn import svm
from Utils import GeoUtils

from scipy import stats
from scipy.stats import norm, skew #for some statistics

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.kernel_ridge import KernelRidge
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from matplotlib.pyplot import figure
from sklearn.decomposition import PCA

%matplotlib inline

In [128]:
def getDistance(real, predict):
    lat1 = real[:,0]
    lon1 = real[:,1]
    lat2 = predict[:,0]
    lon2 = predict[:,1]

    return np.array(GeoUtils.GeoUtils.distanceInKm(lat1, lon1,lat2, lon2))

In [129]:
data = pd.read_csv("Dados/LocTreino_Equipe_4.csv")

bts_data = pd.read_csv("Dados/dados_BTSs.csv")

In [130]:
data.describe()

,pontoId,lat,lon,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1499.000000,1500.000000,1499.000000,1500.000000,1500.0000
mean,1471.292667,-8.070097,-34.894814,116.654370,119.101165,123.863545,115.814052,121.320526,125.656192,0.663333,1.008733,2.525333,1.099388,1.093333,1.0560
std,849.057388,0.004232,0.004354,15.229664,12.666188,9.031316,12.414620,12.552702,13.134365,0.698317,0.864175,0.967117,1.955399,0.856998,0.6692
min,2.000000,-8.077570,-34.906830,86.256667,77.256667,88.340000,84.056667,79.523333,89.856667,0.000000,0.000000,1.000000,0.000000,0.000000,0.0000
25%,746.750000,-8.073197,-34.897387,106.827500,111.673333,119.190000,106.583750,112.823333,116.077500,0.000000,0.000000,2.000000,0.000000,0.000000,1.0000
50%,1464.500000,-8.069940,-34.895346,117.156667,120.690000,124.273333,116.273333,122.840000,128.756667,1.000000,1.000000,3.000000,1.000000,1.000000,1.0000
75%,2190.250000,-8.067113,-34.891169,124.790000,128.808750,130.117083,125.590000,130.423333,135.996250,1.000000,1.000000,3.000000,1.000000,2.000000,2.0000
max,2955.000000,-8.059339,-34.885067,476.882449,144.890000,145.890000,145.423333,146.890000,150.123333,3.000000,11.090452,5.000000,38.747484,3.000000,2.0000


In [131]:
lista = data['pontoId'].tolist()
data.drop_duplicates(inplace=True)
data.shape
data = data.drop(['pontoId'], axis=1)
data['taBTS2'] = data['taBTS2'].fillna(data['taBTS2'].mean())
data['taBTS4'] = data['taBTS4'].fillna(data['taBTS4'].mean())
data.describe()

,lat,lon,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.0000
mean,-8.070097,-34.894814,116.654370,119.101165,123.863545,115.814052,121.320526,125.656192,0.663333,1.008733,2.525333,1.099388,1.093333,1.0560
std,0.004232,0.004354,15.229664,12.666188,9.031316,12.414620,12.552702,13.134365,0.698317,0.863886,0.967117,1.954747,0.856998,0.6692
min,-8.077570,-34.906830,86.256667,77.256667,88.340000,84.056667,79.523333,89.856667,0.000000,0.000000,1.000000,0.000000,0.000000,0.0000
25%,-8.073197,-34.897387,106.827500,111.673333,119.190000,106.583750,112.823333,116.077500,0.000000,0.000000,2.000000,0.000000,0.000000,1.0000
50%,-8.069940,-34.895346,117.156667,120.690000,124.273333,116.273333,122.840000,128.756667,1.000000,1.000000,3.000000,1.000000,1.000000,1.0000
75%,-8.067113,-34.891169,124.790000,128.808750,130.117083,125.590000,130.423333,135.996250,1.000000,1.000000,3.000000,1.000000,2.000000,2.0000
max,-8.059339,-34.885067,476.882449,144.890000,145.890000,145.423333,146.890000,150.123333,3.000000,11.090452,5.000000,38.747484,3.000000,2.0000


In [132]:
x = data.drop(['lat', 'lon'], axis=1)

transformerX = StandardScaler().fit(x)
x_scaled = transformerX.transform(x)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


In [133]:
bts1_coord = np.tile(bts_data[['lat', 'lon']].iloc[0].values, (1500,1))
data['distBTS1'] = getDistance(data[['lat', 'lon']].values, bts1_coord) 

bts2_coord = np.tile(bts_data[['lat', 'lon']].iloc[1].values, (1500,1))
data['distBTS2'] = getDistance(data[['lat', 'lon']].values, bts2_coord) 

bts3_coord = np.tile(bts_data[['lat', 'lon']].iloc[2].values, (1500,1))
data['distBTS3'] = getDistance(data[['lat', 'lon']].values, bts2_coord) 

bts4_coord = np.tile(bts_data[['lat', 'lon']].iloc[3].values, (1500,1))
data['distBTS4'] = getDistance(data[['lat', 'lon']].values, bts2_coord) 

bts5_coord = np.tile(bts_data[['lat', 'lon']].iloc[4].values, (1500,1))
data['distBTS5'] = getDistance(data[['lat', 'lon']].values, bts2_coord) 

bts6_coord = np.tile(bts_data[['lat', 'lon']].iloc[5].values, (1500,1))
data['distBTS6'] = getDistance(data[['lat', 'lon']].values, bts2_coord) 

y = data[['distBTS1', 'distBTS2', 'distBTS3', 'distBTS4', 'distBTS5', 'distBTS6']]
y.describe()

,distBTS1,distBTS2,distBTS3,distBTS4,distBTS5,distBTS6
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
mean,0.651671,0.824295,0.824295,0.824295,0.824295,0.824295
std,0.338396,0.429586,0.429586,0.429586,0.429586,0.429586
min,0.054158,0.009910,0.009910,0.009910,0.009910,0.009910
25%,0.372980,0.527395,0.527395,0.527395,0.527395,0.527395
50%,0.603009,0.760281,0.760281,0.760281,0.760281,0.760281
75%,0.901601,1.023290,1.023290,1.023290,1.023290,1.023290
max,1.847645,2.274950,2.274950,2.274950,2.274950,2.274950


In [134]:
transformerY = StandardScaler().fit(y)
y_scaled = transformerY.transform(y)


X_train, X_test, y_train, y_test = train_test_split(x_scaled, y_scaled, test_size = 0.3)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

invX = transformerX.inverse_transform
invY = transformerY.inverse_transform

((1050, 12), (450, 12), (1050, 6), (450, 6))


In [135]:
knn = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3))
knn.fit(X_train, y_train)

knnreal = invY(y_test)
knnpredict = invY(knn.predict(X_test))

print (np.mean((knnpredict - knnreal)**2, axis=0))

[ 0.00150964  0.0016834   0.0016834   0.0016834   0.0016834   0.0016834 ]


/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and will be removed in 0.13
  ' removed in 0.13', DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and will be removed in 0.13
  ' removed in 0.13', DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and will be removed in 0.13
  ' removed in 0.13', DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and will be removed in 0.13
  ' removed in 0.13', DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and w

In [136]:
d2 = invY(knn.predict(x_scaled))

df2 = pd.DataFrame(d2, columns=['distBTS1', 'distBTS2', 'distBTS3', 'distBTS4', 'distBTS5', 'distBTS6'])

df2.describe()

/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and will be removed in 0.13
  ' removed in 0.13', DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and will be removed in 0.13
  ' removed in 0.13', DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and will be removed in 0.13
  ' removed in 0.13', DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and will be removed in 0.13
  ' removed in 0.13', DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and w

,distBTS1,distBTS2,distBTS3,distBTS4,distBTS5,distBTS6
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
mean,0.652370,0.825186,0.825186,0.825186,0.825186,0.825186
std,0.338194,0.428254,0.428254,0.428254,0.428254,0.428254
min,0.083046,0.028621,0.028621,0.028621,0.028621,0.028621
25%,0.373087,0.524404,0.524404,0.524404,0.524404,0.524404
50%,0.611567,0.760996,0.760996,0.760996,0.760996,0.760996
75%,0.907529,1.016239,1.016239,1.016239,1.016239,1.016239
max,1.844690,2.272323,2.272323,2.272323,2.272323,2.272323


In [137]:
data['distBTS1'] = df2['distBTS1']
data['distBTS2'] = df2['distBTS2']
data['distBTS3'] = df2['distBTS3']
data['distBTS4'] = df2['distBTS4']
data['distBTS5'] = df2['distBTS5']
data['distBTS6'] = df2['distBTS6']
data.describe()

,lat,lon,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6,distBTS1,distBTS2,distBTS3,distBTS4,distBTS5,distBTS6
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.0000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
mean,-8.070097,-34.894814,116.654370,119.101165,123.863545,115.814052,121.320526,125.656192,0.663333,1.008733,2.525333,1.099388,1.093333,1.0560,0.652370,0.825186,0.825186,0.825186,0.825186,0.825186
std,0.004232,0.004354,15.229664,12.666188,9.031316,12.414620,12.552702,13.134365,0.698317,0.863886,0.967117,1.954747,0.856998,0.6692,0.338194,0.428254,0.428254,0.428254,0.428254,0.428254
min,-8.077570,-34.906830,86.256667,77.256667,88.340000,84.056667,79.523333,89.856667,0.000000,0.000000,1.000000,0.000000,0.000000,0.0000,0.083046,0.028621,0.028621,0.028621,0.028621,0.028621
25%,-8.073197,-34.897387,106.827500,111.673333,119.190000,106.583750,112.823333,116.077500,0.000000,0.000000,2.000000,0.000000,0.000000,1.0000,0.373087,0.524404,0.524404,0.524404,0.524404,0.524404
50%,-8.069940,-34.895346,117.156667,120.690000,124.273333,116.273333,122.840000,128.756667,1.000000,1.000000,3.000000,1.000000,1.000000,1.0000,0.611567,0.760996,0.760996,0.760996,0.760996,0.760996
75%,-8.067113,-34.891169,124.790000,128.808750,130.117083,125.590000,130.423333,135.996250,1.000000,1.000000,3.000000,1.000000,2.000000,2.0000,0.907529,1.016239,1.016239,1.016239,1.016239,1.016239
max,-8.059339,-34.885067,476.882449,144.890000,145.890000,145.423333,146.890000,150.123333,3.000000,11.090452,5.000000,38.747484,3.000000,2.0000,1.844690,2.272323,2.272323,2.272323,2.272323,2.272323


In [138]:
x = data.drop(['lat', 'lon'], axis=1)
x.describe()

transformerX = StandardScaler().fit(x)
x_scaled = transformerX.transform(x)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """


In [139]:
y = data[['lat', 'lon']]
y.describe()

,lat,lon
count,1500.000000,1500.000000
mean,-8.070097,-34.894814
std,0.004232,0.004354
min,-8.077570,-34.906830
25%,-8.073197,-34.897387
50%,-8.069940,-34.895346
75%,-8.067113,-34.891169
max,-8.059339,-34.885067


In [140]:
transformerY = StandardScaler().fit(y)
y_scaled = transformerY.transform(y)


X_train, X_test, y_train, y_test = train_test_split(x_scaled, y_scaled, test_size = 0.3)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

invX = transformerX.inverse_transform
invY = transformerY.inverse_transform

((1050, 18), (450, 18), (1050, 2), (450, 2))


In [141]:
knn = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3))
knn.fit(X_train, y_train)

knnreal = invY(y_test)
knnpredict = invY(knn.predict(X_test))

print(getDistance(knnreal, knnpredict).mean())

0.0370855186963


/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and will be removed in 0.13
  ' removed in 0.13', DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and will be removed in 0.13
  ' removed in 0.13', DeprecationWarning)


In [142]:
svr = svm.SVR()
regr = MultiOutputRegressor(svr)
regr.fit(X_train,y_train)

svmreal = invY(y_test)
svmpredict = invY(regr.predict(X_test))

print(getDistance(svmreal, svmpredict).mean())

0.0537967701522


In [143]:
import math
import mpu

L = 3
B = 6

RES_list = []
RES_list.append(['pontoId', 'lat','lon','lat_pred','lon_pred','erro_loc'])

BTS_mtz = np.array([[1.]*L]*B)

def coord_conv(lat,lon,alt):
	lat = float(lat)
	lon = float(lon)
	earthR = 6371
	x = earthR *(math.cos(math.radians(lat)) * math.cos(math.radians(lon)))
	y = earthR *(math.cos(math.radians(lat)) * math.sin(math.radians(lon)))
	z = earthR *(math.sin(math.radians(lat)))

	return np.array([x,y,z])

def km_dist(p, lat, lon):
	return mpu.haversine_distance((data['lat'][p], data['lon'][p]) , (lat,lon))

def initalize_bts():
    BTS_mtz[0] = coord_conv(bts1_coord[0][0], bts1_coord[0][1],0)
    BTS_mtz[1] = coord_conv(bts2_coord[0][0], bts2_coord[0][1],0)
    BTS_mtz[2] = coord_conv(bts3_coord[0][0], bts3_coord[0][1],0)
    BTS_mtz[3] = coord_conv(bts4_coord[0][0], bts4_coord[0][1],0)
    BTS_mtz[4] = coord_conv(bts5_coord[0][0], bts5_coord[0][1],0)
    BTS_mtz[5] = coord_conv(bts6_coord[0][0], bts6_coord[0][1],0)
    


def tri_lat(p_index,i_index,j_index,k_index, dist_mtz):
	#print p_index, i_index, j_index, k_index
	# convert_np() #initializate BTS_mtz
	# dist_mtz = dist_array() #m


	P1 = np.copy(BTS_mtz[i_index])
	P2 = np.copy(BTS_mtz[j_index])
	P3 = np.copy(BTS_mtz[k_index])

	Dist1 = np.copy(df2['distBTS'+str(i_index+1)][p_index])
	Dist2 = np.copy(df2['distBTS'+str(j_index+1)][p_index])
	Dist3 = np.copy(df2['distBTS'+str(k_index+1)][p_index])

	#Translade circles references (BTS)
	ex = (P2 - P1)/(np.linalg.norm(P2 - P1))
	i = np.dot(ex,P3 - P1)
	ey = (P3 - P1 - i*ex)/(np.linalg.norm(P3 - P1 - i*ex))
	ez = np.cross(ex,ey)
	d = np.linalg.norm(P2 - P1)
	j = np.dot(ey,P3 - P1)

	#plug and chug
	x = (pow(Dist1,2) - pow(Dist2,2) + pow(d,2))/(2*d)
	y = (( pow(Dist1,2) - pow(Dist3,2) + pow(i,2) + pow(j,2))/(2*j))
	- ((i/j) *x)

	#only contendoa
	test = (pow(Dist1,2) - pow(x,2) - pow(y,2))
	if(test < 0):
		return (float("inf"),0,0)
	else:
		z = np.sqrt(test)


	triPt = P1 + x*ex + y*ey + z*ez

	#Back
	earthR = 6371
	if(triPt[2]/earthR < -1 or triPt[2]/earthR > 1):
		# print "Forced again"
		# return float("inf")
		return (float("inf"),0,0)
	lat = math.degrees(math.asin(triPt[2] / earthR))
	lon = math.degrees(math.atan2(triPt[1],triPt[0]))

	dist = km_dist(p_index,lat,lon)
	return (dist,lat,lon)


#Menor erro com determinada BTS
def min_error(p_index, bst_index, dist_mtz):
	list = []
	for i in range (0,6):
		if(i!=bst_index):
			for j in range (0,6):
				if(j!=bst_index and j!= i):
					list.append( tri_lat(p_index,bst_index,i,j,dist_mtz))
					# print "i,j,k -", bst_index,i,j
					#print "C,I,J: ",bst_index, i,j
	#print "Min Dist - ", min(list)*1000, "m"
	return min(list)

#Menor erro com combinacao de todas as BTSs
def best_error(p_index,dist_mtz):
	list = []
	for i in range (0,6):
		list.append(min_error(p_index,i,dist_mtz))

	return min(list)

P = 1500
def all_points(dist_mtz):
	sum = 0
	p = 1500
	for i in range(0,P):
		RES_list.append([lista[i]])    
		(err,lat,lon) = best_error(i,dist_mtz)
		RES_list[i+1].append(data['lat'][i])
		RES_list[i+1].append(data['lon'][i])
		RES_list[i+1].append(lat)
		RES_list[i+1].append(lon)
		RES_list[i+1].append(err*1000)
		# print "Point:", i, " Min Error ", err*1000, "m"
		# print "Real: ", LOC_mtz[i][0],",",LOC_mtz[i][1]
		# print "Pred: ", lat,",",lon, "\n"
		if(err != float("inf")):
			sum += err
		else:
			p = p-1
	print "Average error - ", (sum/p)*1000, "m" , "with:", p ," points"          

In [144]:
initalize_bts()
print BTS_mtz

df2.describe()

print df2['distBTS1'][0]

Dist1 = np.copy(df2['distBTS'+str(1)][1499])

data.describe()

all_points(df2)

#print RES_list



[[ 5173.92348894 -3608.40207688  -894.19862566]
 [ 5173.70776525 -3608.50515716  -895.0304411 ]
 [ 5172.85869409 -3609.7100785   -895.07937088]
 [ 5173.70776522 -3608.50515722  -895.0304411 ]
 [ 5174.16013139 -3608.12715558  -893.93868017]
 [ 5173.85462323 -3608.60385761  -893.78271214]]
0.872493444497
Average error -  245.696733472 m with: 1500  points
